# TelecomX_BR Parte 2 - Criação de um Modelo para identificar a evasão de cliêntes

# Tratamento dos dados


### 📥 Importação de Bibliotecas


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

### 📂 Carregamento e Visualização Inicial dos Dados

In [ ]:
# Lendo os dados
dados = pd.read_csv('G:/Meu Drive/Oracle_alura/Estatistica_ML/Challenge_3/dados/dados_tratados.csv')
print(dados.info())

### 🧹 Limpeza de Dados

In [ ]:
#Removendo valores nulos
dados.fillna(dados.mean(numeric_only=True), inplace=True)

#Removendo colunas inuteis
dados.drop(columns='customerID', inplace=True)

### 🔁 Encoding de Variáveis Binárias

In [ ]:
#Realizando o encoding das variaveis
print(dados.nunique())

#Convertendo as variaveis explicativas binarias
dados['Churn'].replace({'No': 0, 'Yes': 1},inplace=True)
dados['customer.gender'].replace({'Female': 0, 'Male': 1}, inplace=True)
dados['customer.Partner'].replace({'No': 0, 'Yes': 1}, inplace=True)
dados['customer.Dependents'].replace({'No': 0, 'Yes': 1}, inplace=True)
dados['phone.PhoneService'].replace({'No': 0, 'Yes': 1}, inplace=True)
dados['account.PaperlessBilling'].replace({'No': 0, 'Yes': 1}, inplace=True)

#Criando uma coluna pra quem tem ou não serviço se phone ou internete, transformando as originais em binarias
dados['has.PhoneService'] = dados['phone.MultipleLines'].map(lambda x: 0 if x=='No phone service' else 1)
dados['phone.MultipleLines'].replace({'No': 0, 'Yes': 1, 'No phone service': 0},inplace=True)

colunas = ['internet.OnlineSecurity','internet.OnlineBackup','internet.DeviceProtection','internet.TechSupport','internet.StreamingTV','internet.StreamingMovies']
dados[colunas]=dados[colunas].replace({'No': 0, 'Yes': 1, 'No internet service': 0})
dados['has.InternetService'] = dados[colunas].apply(lambda linha: 0 if all(valor == 0 for valor in linha) else 1, axis=1)
print(dados.info())

### 🧠 One-Hot Encoding para Variáveis Não Binárias

In [ ]:
#Convertendo as explicativas nao binarias utilizando o One hot encoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

#Separando variaveis explicativas da independente
X  = dados.drop(columns='Churn')
Y  = dados['Churn']
#Salvando as colunas originais
colunas = X.columns
one_hot = make_column_transformer((OneHotEncoder(drop='if_binary'),['account.Contract','account.PaymentMethod','internet.InternetService']),remainder='passthrough',sparse_threshold=0)

#Aplicando a transformação nas variaveis explicativas
X = one_hot.fit_transform(X)
X = pd.DataFrame(X, columns=one_hot.get_feature_names_out(colunas))
print(Y)
print(X.info())

### 🔍 Análise de Correlação com a Variável Churn

In [ ]:
#Observando a proporção de evasão da variavel dependente
print(Y.value_counts(normalize=True))
#73.5% não saiu, 26.5% saiu


#Realizando a análise da correlação dos parâmetros
correlacoes = {}
for coluna in X.columns:
    correlacoes[coluna] = pd.Series(X[coluna]).corr(Y)

# Convertendo para DataFrame para facilitar a visualização
correlacoes_df = pd.DataFrame.from_dict(correlacoes, orient='index', columns=['Correlação com Churn'])
correlacoes_df = correlacoes_df.sort_values(by='Correlação com Churn', ascending=False)

# Plotando as correlações
plt.figure(figsize=(10, 8))
sns.barplot(x=correlacoes_df['Correlação com Churn'], y=correlacoes_df.index, palette='coolwarm')
plt.title('Correlação das variáveis com Churn')
plt.xlabel('Correlação')
plt.ylabel('Variável')
plt.tight_layout()
plt.show()

![Correlação das variáveis com Churn](<../gráficos/correlação com churn.png>)

### ⚖️ Normalização dos Dados com MinMaxScaler

In [ ]:
#Fazendo dados normalizados
from sklearn.preprocessing import MinMaxScaler
#Inicializando o objeto da normalização em uma variável
normalizacao = MinMaxScaler()
colunas = X.columns

#Aplicando a transformação nos dados de treinamento com a função .fit_transform()
X_normal = pd.DataFrame(normalizacao.fit_transform(X), columns=colunas)

#Visualizando as informações normalizadas
fig = px.histogram(X_normal, x='onehotencoder__account.Contract_Month-to-month', text_auto=True, color=Y, barmode='group')
fig.show()

![Dados Normalizados](<../gráficos/dados normalizados.png>)

# Modelo RandomForestClassifier

### 📥 Importação de Bibliotecas e Dados Tratados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from Tratamento import X, Y

### 📊 Divisão dos Dados e Balanceamento com SMOTE

In [ ]:
#Realizando a divisão dos dados com 80% usado para treino
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.20)


#Balanceamento apenas dos dados de treino com SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

### 🌲 Criação e Avaliação Inicial do Modelo Random Forest

In [ ]:
#Criando o modelo de arvore e utilizando a validação cruzada
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold

#Criando um modelo
floresta = RandomForestClassifier(max_depth = 10, random_state=5)
#Aplicando o floresta
floresta.fit(X_train, Y_train)
print('\nDesempenho do modelo' ,floresta.score(X_test, Y_test))

### 🔁 Validação Cruzada com Pipeline + SMOTE

In [ ]:
#Validação cruzada com pipeline
from imblearn.pipeline import Pipeline as imbpipeline

pipeline = imbpipeline([
    ('smote', SMOTE(random_state=5)),
    ('modelo', RandomForestClassifier(max_depth=10, random_state=5))
])
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=5)
cv_resultados = cross_validate(pipeline, X, Y, cv=skf, scoring='f1')
print('\nResultados da validação cruzada\n', cv_resultados)

#Criando uma função para obter o intervalo de confiança
def intervalo_conf(resultados):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print(f'\nIntervalo de confiança: [{media - 2*desvio_padrao}, {min(media + 2*desvio_padrao, 1)}]')
intervalo_conf(cv_resultados)

### 🧪 Métricas de Avaliação do Modelo

In [ ]:
#Observando as métricas do modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
Y_prev = floresta.predict(X_test)
print('Acurácia: ', accuracy_score(Y_test, Y_prev))
print('Precisão: ', precision_score(Y_test, Y_prev))
print('Recall: ', recall_score(Y_test, Y_prev))
print('F1 Score: ', f1_score(Y_test, Y_prev))

#Construindo a curva a partir das previsões feitas pelo modelo
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(Y_test, Y_prev, name='Arvore de decisão')
plt.show()

![Curva ROC](<../gráficos/curva ROC.png>)

In [ ]:
#Avaliando com a matriz de confusão
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

#Criando a matriz
matriz_confusao = confusion_matrix(Y_test, Y_prev)
#Visualizando a matriz de confusão
visualizacao = ConfusionMatrixDisplay(matriz_confusao, display_labels=['Deixou o serviço', 'Não deixou o serviço'])
visualizacao.plot()
plt.show()

![Matriz de confusão 1](<../gráficos/matriz de confusão 1.png>)

### 🧠 Importância das Features

In [ ]:
#Entendendo agora as features mais importantes do modelo de floresta, as 10 primeiras
from yellowbrick.model_selection import FeatureImportances
viz =  FeatureImportances(floresta, relative=False)#O parâmetro relative=False define a importancia absoluta
viz.fit(X_train, Y_train)
viz.show()

importances = floresta.feature_importances_
#Transformando em um data frame e ordenando os valores para melhorar a visualização
feature_importances = pd.DataFrame({'Features': X.columns, 'Importances': importances})
feature_importances.sort_values('Importances', ascending=False, inplace=True)
print('Importancia de cada feature',feature_importances)

![Matriz de confusão 1](<../gráficos/features importantes.png>)

In [ ]:
# Função para calcular as métricas de classificação
def calcular_metricas_classificacao(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    
    metricas = {
        'Acurácia': round(acc, 4),
        'Precisão': round(prec, 4),
        'Recall': round(rec, 4),
        'F1 Score': round(f1, 4)
    }
    return metricas

# Redeterminando o modelo com base nas features mais importantes
# Criando um data frame onde os índices são as métricas
results_df = pd.DataFrame(index=['Acurácia', 'Precisão', 'Recall', 'F1 Score'])

# Recriando o modelo da floresta
model_selected_features = RandomForestClassifier(random_state=5, max_depth=5)

# Criando um loop, para pegar as 1 a 19 features mais importantes
for count in range(1, 20):
    # Selecionando os nomes das features com base no data frame das features mais importantes
    selected_features = feature_importances['Features'].values[:count]
    # Pegando os dados de treino e teste de acordo com as features selecionadas
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]
    # Ajustando o modelo com esses novos dados de treino
    model_selected_features.fit(X_train_selected, Y_train)
    # Fazendo a previsão do modelo com base nos novos dados de teste
    Y_pred = model_selected_features.predict(X_test_selected)
    # Calculando as novas métricas
    metricas = calcular_metricas_classificacao(Y_test, Y_pred)
    # Colocando o resultado das métricas em um data frame
    results_df[count] = list(metricas.values())
print('\nResultados:\n', results_df)

### ✅ Reavaliação com as 17 Melhores Features

In [ ]:
#Mantendo apenas as 17 primeiras features
selected_features = feature_importances['Features'].values[:17]
X_select_features = X[selected_features]


#Separando os dados de treino e teste apenas com essas features
X_train, X_test, Y_train, Y_test = train_test_split(X_select_features, Y, random_state=5, test_size=0.20)

### 🔍 Otimização de Hiperparâmetros com GridSearchCV

In [ ]:
#Recriando o pipeline
pipeline = imbpipeline([
    ('smote', SMOTE(random_state=5)),
    ('modelo', RandomForestClassifier(random_state=5, class_weight={0: 2, 1: 1}))
])

#Fazendo o modelo só que com os hiperparâmetros
#Grid dos parâmetros do RandomForestClassifier
param_grid = {
    'modelo__n_estimators': [100, 150, 200],
    'modelo__max_depth': [5, 10, 15],
    'modelo__min_samples_split': [2, 4, 6],
    'modelo__min_samples_leaf': [1, 2, 3]
}

#Importando o GridSearchCV e criando o modelo
from sklearn.model_selection import GridSearchCV
floresta_grid = GridSearchCV(pipeline, param_grid=param_grid, scoring='f1', cv=skf)

#Treinando o modelo
floresta_grid.fit(X_train, Y_train)
#Visualizando os melhores parâmetros com o metodo .best_params_
print(floresta_grid.best_params_)

### 📋 Avaliação Final do Modelo Otimizado

In [ ]:
#Observando novamente as métricas
Y_prev = floresta_grid.predict(X_test)
print('Acurácia: ', accuracy_score(Y_test, Y_prev))
print('Precisão: ', precision_score(Y_test, Y_prev))
print('Recall: ', recall_score(Y_test, Y_prev))
print('F1 Score: ', f1_score(Y_test, Y_prev))

#visualizando a matriz de confusão de novo
matriz_confusao = confusion_matrix(Y_test, Y_prev)
visualizacao = ConfusionMatrixDisplay(matriz_confusao, display_labels=['Deixou o serviço', 'Não deixou o serviço'])
visualizacao.plot()
plt.show()

![Matriz de confusão 2](<../gráficos/matriz de confusão 2.png>)

# Modelo KNN

### 📥 Importação de Bibliotecas e Dados

In [ ]:
import matplotlib.pyplot as plt
from Tratamento import Y, X_normal

### 📊 Divisão dos Dados Normalizados e Balanceamento com SMOTE

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_normal, Y, test_size= 0.20)


#Balanceamento apenas dos dados de treino com SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

### 🔍 Busca do Melhor K para o Modelo KNN (GridSearchCV)

In [ ]:
#Utilizando o algoritimo KNN da biblioteca sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
#Procurando o melhor valor para K, e o melhor F1 desse valor
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1')
grid.fit(X_train, Y_train)

print('\nMelhor valor de K:\n', grid.best_params_)
print('\nMelhor Recall score:\n', grid.best_score_)

#Criando o modelo de fato com o melhor K descoberto pelo grid
#Inicializando o algoritimo em uma variavel
knn = grid.best_estimator_
#Ajustando o modelo aos dados
knn.fit(X_train, Y_train)

### 📋 Avaliação do Modelo KNN

In [ ]:
#Observando as métricas do modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
Y_prev = knn.predict(X_test)
print('\nAcurácia: ', accuracy_score(Y_test, Y_prev))
print('Precisão: ', precision_score(Y_test, Y_prev))
print('Recall: ', recall_score(Y_test, Y_prev))
print('F1 Score: ', f1_score(Y_test, Y_prev))


#Construindo a curva a partir das previsões feitas pelo modelo
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(Y_test, Y_prev, name='KNN')
plt.show()


#Avaliando com a matriz de confusão
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

#Criando a matriz
matriz_confusao = confusion_matrix(Y_test, Y_prev)
#Visualizando a matriz de confusão
visualizacao = ConfusionMatrixDisplay(matriz_confusao, display_labels=['Deixou o serviço', 'Não deixou o serviço'])
visualizacao.plot()
plt.show()

#Foi considerado que esse modelo é bem ruim, nem vale a pena ajustar hiperparâmetros


![Matriz de confusão KNN](<../gráficos/matriz de confusão knn.png>)

# Relatório Final

## 📘 Introdução

Este relatório apresenta os resultados da modelagem preditiva aplicada ao problema de **evasão de clientes da Telecom X**. O principal objetivo foi desenvolver modelos capazes de prever quais clientes têm maior chance de cancelar seus serviços. As etapas envolvidas incluíram o tratamento e preparação dos dados, treinamento de diferentes modelos, análise de desempenho e identificação dos fatores mais influentes na evasão.

## 🛠️ Metodologia

### Pré-processamento

- Tratamento de valores nulos: Substituição por média (para variáveis numéricas).

- Encoding de variáveis:
    - Binárias: substituídas por 0 e 1.
    - Categóricas com múltiplos valores: transformadas por One-Hot Encoding.

- Criação de novas variáveis:
    - has.PhoneService e has.InternetService: sinalizam se o cliente possui algum serviço básico ativo.

- Normalização: Aplicada via MinMaxScaler para modelos sensíveis à escala.

- Balanceamento da base de treino: Realizado com SMOTE, para lidar com a desproporção entre classes (73,5% não saíram vs. 26,5% que saíram).

### Modelos treinados

Com base na matriz de confusão, foi decidido que a métrica à ser priorizada seria o Recall, que representa a taxa de verdadeiros positivos que foram classificados como positivos mesmo, visto que a taxa de dados que foi decidida minimizar foi a dos falsos negativos.

- Modelo 1 – KNN (K-Nearest Neighbors)
    Melhor K: 5

    Métricas: Acurácia: 
    - **74%** / Precisão: **57%** / Recall: **53%** / F1 Score: **55%**

Conclusão: Modelo com desempenho insatisfatório, especialmente no recall (foco principal).

- Modelo 2 - Random Forest Classifier
    Parâmetros ajustados via GridSearch: 
    - n_estimators: **150** / max_depth: **10** / min_samples_split: **2** / min_samples_leaf: **2**


    Class weight ajustado para reduzir falsos negativos: {0: 2, 1: 1}


    Métricas com 17 features mais importantes: Acurácia: 
    - **80%** / Precisão: **66%** / Recall: **72%** / F1 Score: **69%**

Conclusão: Melhor modelo para o objetivo do problema (maximizar o recall).

## 🔍 Análise de Importância das Variáveis

- As 10 variáveis mais relevantes no modelo de Floresta foram:

| Rank | Variável                               | Descrição                                                  |
| ---- | -------------------------------------- | ---------------------------------------------------------- |
| 1    | `account.Contract_Month-to-month`      | Contrato mensal: forte indicativo de churn                 |
| 2    | `account.tenure`                       | Tempo como cliente: quanto menor, maior o risco            |
| 3    | `account.MonthlyCharges`               | Cobrança mensal: valores altos correlacionam com evasão    |
| 4    | `has.InternetService`                  | Clientes com internet são mais sensíveis ao serviço        |
| 5    | `internet.InternetService_Fiber optic` | Clientes de fibra óptica apresentaram maior evasão         |
| 6    | `account.PaperlessBilling`             | Cobrança sem papel aumenta risco de evasão                 |
| 7    | `internet.OnlineSecurity`              | Clientes com segurança online tendem a permanecer mais     |
| 8    | `customer.Partner`                     | Clientes sem parceiro têm maior probabilidade de sair      |
| 9    | `account.TotalCharges`                 | Menores valores totais acumulados sugerem menos fidelidade |
| 10   | `internet.TechSupport`                 | Suporte técnico reduz a chance de evasão                   |


- Com base na análise dos gráficos de correlação e nas variáveis mais relevantes identificadas pelo modelo preditivo, observou-se que é fundamental priorizar a atenção aos clientes com menor tempo de vínculo com a empresa. Clientes com contratos do tipo mês a mês apresentam uma propensão significativamente maior à evasão, uma vez que possuem maior liberdade e facilidade para cancelar o serviço a qualquer momento.

- Esse padrão é reforçado pela influência direta do tempo total de contrato (tenure) sobre o churn: quanto menor o tempo de permanência, maior o risco de cancelamento. Isso indica que os primeiros meses da jornada do cliente são críticos e exigem ações estratégicas para aumentar o engajamento e a fidelização.

- Outro fator de destaque é o valor da cobrança mensal, que demonstrou forte correlação com os contratos de curta duração. Cobranças elevadas, especialmente em contratos mensais, podem contribuir para a insatisfação e impulsionar a decisão de cancelamento. Portanto, ofertas mais flexíveis e ajustadas ao perfil de consumo desses clientes podem ser uma medida eficaz para reduzir a evasão.


## 🎯 Estratégias de Retenção com Base nos Resultado

- **Campanhas de incentivo à migração para contratos de longo prazo** (trimestral ou anual), oferecendo descontos progressivos ou bônus (como dados extras, meses gratuitos ou upgrades de serviço).

- **Comunicação segmentada com foco nos benefícios da fidelidade**, segurança e estabilidade de preços em contratos mais longos.

- **Implantar um programa de onboarding personalizado**, com suporte técnico inicial e materiais educativos sobre o uso do serviço.

- **Oferecer benefícios exclusivos para pagamentos antecipados ou combo de serviços**, agregando valor à experiência e diluindo a percepção de custo elevado.

- **Realizar contato proativo nos primeiros 30 dias**, via e-mail ou ligação, para garantir satisfação e resolver eventuais problemas rapidamente.
